In [ ]:
#!/usr/bin/env python3
# Copyright (c) Meta Platforms, Inc. and affiliates.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""
Runs MNIST training with differential privacy.

"""

import argparse

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from opacus import PrivacyEngine
from torchvision import datasets, transforms
from tqdm import tqdm


# Precomputed characteristics of the MNIST dataset
MNIST_MEAN = 0.1307
MNIST_STD = 0.3081
delta = 1e-5

In [ ]:
# class SampleConvNet(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Conv2d(1, 16, 8, 2, padding=3)
#         self.conv2 = nn.Conv2d(16, 32, 4, 2)
#         self.fc1 = nn.Linear(32 * 4 * 4, 32)
#         self.fc2 = nn.Linear(32, 10)

#     def forward(self, x):
#         # x of shape [B, 1, 28, 28]
#         x = F.relu(self.conv1(x))  # -> [B, 16, 14, 14]
#         x = F.max_pool2d(x, 2, 1)  # -> [B, 16, 13, 13]
#         x = F.relu(self.conv2(x))  # -> [B, 32, 5, 5]
#         x = F.max_pool2d(x, 2, 1)  # -> [B, 32, 4, 4]
#         x = x.view(-1, 32 * 4 * 4)  # -> [B, 512]
#         x = F.relu(self.fc1(x))  # -> [B, 32]
#         x = self.fc2(x)  # -> [B, 10]
#         return x

#     def name(self):
#         return "SampleConvNet"


class MnistNet(nn.Module):
    """Mnist network definition."""

    def __init__(self) -> None:
        """Initialization of the network."""
        super().__init__()
        self.fc1 = nn.Linear(28 * 28, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 10)

    def forward(self, input_data):
        """Defines the forward pass of the network.

        Args:
            input_data (Tensor): Input data

        Returns
        -------
            Tensor: Output data
        """
        out = input_data.view(-1, 28 * 28)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        return self.fc3(out)

In [ ]:
def train(model, device, train_loader, optimizer, privacy_engine, epoch):
    model.train()
    criterion = nn.CrossEntropyLoss()
    losses = []
    correct = 0
    for _batch_idx, (data, target) in enumerate(tqdm(train_loader)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        pred = output.argmax(
            dim=1, keepdim=True
        )  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

    epsilon = privacy_engine.accountant.get_epsilon(delta=delta)
    print(
        f"Train Epoch: {epoch} \t"
        f"Loss: {np.mean(losses):.6f} "
        f"(ε = {epsilon:.2f}, δ = {delta})"
        f"(Accuracy: {100.0 * correct / len(train_loader.dataset)})"
    )

In [ ]:
def test(model, device, test_loader):
    model.eval()
    criterion = nn.CrossEntropyLoss()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in tqdm(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(
                dim=1, keepdim=True
            )  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n".format(
            test_loss,
            correct,
            len(test_loader.dataset),
            100.0 * correct / len(test_loader.dataset),
        )
    )
    return correct / len(test_loader.dataset)

In [ ]:
device = torch.device("cuda:0")
data_root = "../mnist"
test_batch_size = 256
batch_size = 256
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        data_root,
        train=True,
        download=True,
        transform=transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize((MNIST_MEAN,), (MNIST_STD,)),
            ]
        ),
    ),
    batch_size=batch_size,
    num_workers=0,
    pin_memory=True,
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        data_root,
        train=False,
        transform=transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize((MNIST_MEAN,), (MNIST_STD,)),
            ]
        ),
    ),
    batch_size=test_batch_size,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
)

In [ ]:
run_results = []
epochs = 10
lr = 0.01
sigma = 2.0
max_per_sample_grad_norm = 5.0
model = MnistNet().to(device)

optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0)

privacy_engine = PrivacyEngine(accountant="rdp")
model, optimizer, train_loader = privacy_engine.make_private(
    module=model,
    optimizer=optimizer,
    data_loader=test_loader,
    noise_multiplier=sigma,
    max_grad_norm=max_per_sample_grad_norm,
)

In [ ]:
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, privacy_engine, epoch)

In [ ]:
import sys


def train_process(queue, weights, train_loader, privacy_engine, epoch):
    print(f"Training epoch: {epoch}")

    def get_parameters(model):
        return [val.cpu().numpy() for _, val in model.state_dict().items()]

    def set_params(model: torch.nn.ModuleList, params):
        """Set model weights from a list of NumPy ndarrays."""
        params_dict = zip(model.state_dict().keys(), params)
        state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
        model.load_state_dict(state_dict, strict=True)

    model = MnistNet()
    set_params(model, weights)
    model.to("cuda:0")
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0)

    privacy_engine = PrivacyEngine(accountant="rdp")
    model, optimizer, train_loader = privacy_engine.make_private(
        module=model,
        optimizer=optimizer,
        data_loader=test_loader,
        noise_multiplier=sigma,
        max_grad_norm=max_per_sample_grad_norm,
    )

    model.train()
    criterion = nn.CrossEntropyLoss()
    losses = []
    correct = 0
    print(f"Starting Train: {epoch}")
    sys.stdout.flush()
    for _batch_idx, (data, target) in enumerate(tqdm(train_loader)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        pred = output.argmax(
            dim=1, keepdim=True
        )  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

    epsilon = privacy_engine.accountant.get_epsilon(delta=delta)
    print(
        f"Train Epoch: {epoch} \t"
        f"Loss: {np.mean(losses):.6f} "
        f"(ε = {epsilon:.2f}, δ = {delta})"
        f"(Accuracy: {100.0 * correct / len(train_loader.dataset)})"
    )
    sys.stdout.flush()
    queue.put((get_parameters(model), privacy_engine))

In [ ]:
from multiprocess import Process, Queue
from collections import OrderedDict


def get_parameters(model):
    return [val.cpu().numpy() for _, val in model.state_dict().items()]


def set_params(model: torch.nn.ModuleList, params):
    """Set model weights from a list of NumPy ndarrays."""
    params_dict = zip(model.state_dict().keys(), params)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    model.load_state_dict(state_dict, strict=True)


model = MnistNet().to(device)

weights = get_parameters(model)
privacy_engine = PrivacyEngine(accountant="rdp")

q = Queue()
p = Process(target=train_process, args=[q, weights, train_loader, privacy_engine, 0])
p.start()
weights, privacy_engine = q.get()
p.join()

set_params(model, weights)

weigts = get_parameters(model)

q = Queue()
p = Process(target=train_process, args=[q, weigts, device, privacy_engine, 1])
p.start()
weights, privacy_engine = q.get()
p.join()

set_params(model, weights)

weigts = get_parameters(model)

q = Queue()
p = Process(target=train_process, args=[q, weigts, device, privacy_engine, 1])
p.start()
weights, privacy_engine = q.get()
p.join()

set_params(model, weights)

weigts = get_parameters(model)

In [1]:
from torchvision import models
from torchsummary import summary

from pistacchio_simulator.Models.celeba import CelebaNet
import torch 
model = CelebaNet()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
summary(model, input_size=(3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 64, 64]             224
              ReLU-2            [-1, 8, 64, 64]               0
         MaxPool2d-3            [-1, 8, 32, 32]               0
            Conv2d-4           [-1, 16, 32, 32]           1,168
              ReLU-5           [-1, 16, 32, 32]               0
         MaxPool2d-6           [-1, 16, 16, 16]               0
            Conv2d-7           [-1, 16, 16, 16]           2,320
              ReLU-8           [-1, 16, 16, 16]               0
         MaxPool2d-9             [-1, 16, 8, 8]               0
           Linear-10                    [-1, 4]           4,100
Total params: 7,812
Trainable params: 7,812
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.05
Forward/backward pass size (MB): 0.91
Params size (MB): 0.03
Estimated Total